# Caterva - PyData NYC 2019

In [1]:
import os
import cat4py as cat
import numpy as np
from time import time

## NPArray

In [2]:
shape = (3000, 3000)
chunkshape = (1000, 1200)
blockshape = (300, 25)
dtype = 'f8'

a = np.linspace(0, 1, np.prod(shape), dtype=dtype).reshape(shape)

### Different ways to create a Caterva NPArray

In [3]:
b1 = cat.from_numpy(a, chunkshape=chunkshape, blockshape=blockshape)

AttributeError: module 'cat4py' has no attribute 'from_numpy'

In [ ]:
b2 = cat.empty(shape, dtype=dtype, chunkshape=chunkshape, blockshape=blockshape)
for block, info in b2.iter_write():
    block[:] = a[info.slice]

In [ ]:
b3 = cat.from_buffer(bytes(a), shape, dtype=dtype, chunkshape=chunkshape, blockshape=blockshape)

### Read iterator over Caterva NPArray

In [ ]:
for block, info in b3.iter_read():
    np.testing.assert_allclose(block, a[info.slice])

### Getting a slice from a Caterva NPArray

In [ ]:
b2[3:40, 200:500]

### Serialize Caterva NPArray

In [ ]:
c1 = cat.from_numpy(a, chunkshape=chunkshape, blockshape=blockshape)

t0 = time()
sframe = c1.to_sframe()
t1 = time()
tnsf = t1 - t0

print(f"No serialized format: {tnsf:.4f}")

In [ ]:
c2 = cat.from_numpy(a, chunkshape=chunkshape, blockshape=blockshape, enforceframe=True)

t0 = time()
sframe = c2.to_sframe()
t1 = time()
tsf = t1 - t0

print(f"Serialized format: {tsf:.4f}")

In [ ]:
print(f"Speed-up: {(tnsf / tsf):.4f}")

### Persistency

In [ ]:
if os.path.exists("caterva-demo.cat"):
    os.remove("caterva-demo.cat")
    
d1 = cat.from_numpy(a, chunkshape=chunkshape, blockshape=blockshape, filename="caterva-demo.cat")
d2 = cat.from_file("caterva-demo.cat")